In [1]:
import prior

dataset = prior.load_dataset("procthor-10k")
dataset

ModuleNotFoundError: No module named 'prior'

In [3]:
house = dataset["train"][0]
type(house), house.keys(), house

(dict,
 dict_keys(['doors', 'metadata', 'objects', 'proceduralParameters', 'rooms', 'walls', 'windows']),
 {'doors': [{'assetId': 'Doorway_6',
    'id': 'door|1|2',
    'openable': False,
    'openness': 0,
    'room0': 'room|2',
    'room1': 'room|2',
    'wall0': 'wall|2|5.99|3.99|5.99|5.99',
    'wall1': 'wall|exterior|5.99|3.99|5.99|5.99',
    'holePolygon': [{'x': 0.4987163812589276, 'y': 0, 'z': 0},
     {'x': 1.5362947256040203, 'y': 2.1068506240844727, 'z': 0}],
    'assetPosition': {'x': 1.016530884310268,
     'y': 1.0534253120422363,
     'z': 0}}],
  'metadata': {'agent': {'horizon': 30,
    'position': {'x': 3.5, 'y': 0.95, 'z': 2.0},
    'rotation': {'x': 0, 'y': 90, 'z': 0},
    'standing': True},
   'roomSpecId': 'kitchen',
   'schema': '1.0.0',
   'warnings': {},
   'agentPoses': {'arm': {'horizon': 30,
     'position': {'x': 3.5, 'y': 0.95, 'z': 2.0},
     'rotation': {'x': 0, 'y': 90, 'z': 0},
     'standing': True},
    'default': {'horizon': 30,
     'position': {'

In [5]:
from ai2thor.controller import Controller
house = dataset["train"][3]
controller = Controller(scene=house)

In [6]:
import ai2thor.controller
import numpy as np
import random
import time
from PIL import Image
import os

# Initialize ProcTHOR controller
controller = ai2thor.controller.Controller(scene=house)

# Step 1: Get all reachable positions
event = controller.step("GetReachablePositions")
print(event.metadata)
reachable_positions = event.metadata["actionReturn"]
print(f"Reachable positions: {len(reachable_positions)}")

# Helper: discretize position for coverage grid
def discretize(pos, cell_size=0.25):
    return (round(pos["x"]/cell_size), round(pos["z"]/cell_size))

reachable_set = {discretize(p) for p in reachable_positions}
visited = set()

# Step 2: Main exploration loop
coverage_history = []
step = 0
max_steps = 100

def get_agent_position():
    pos = controller.last_event.metadata["agent"]["position"]
    return discretize(pos)

def save_img(i, event):
    img = Image.fromarray(event.frame)
    img.save(f"data/images/frame_{i}.png")
    return i + 1

os.makedirs("data/images", exist_ok=True)

i = 0
for step in range(max_steps):
    pos = get_agent_position()
    visited.add(pos)

    # Compute coverage
    coverage = len(visited) / len(reachable_set)
    coverage_history.append(coverage)
    print(f"[Step {step}] Coverage: {coverage:.2%}")

    # Check if we’ve seen everything
    if coverage >= 0.99:
        print("✅ Full coverage achieved!")
        break

    # Pick nearest unvisited cell
    unvisited = [p for p in reachable_positions if discretize(p) not in visited]
    if not unvisited:
        print("No unvisited cells left.")
        break

    # Pick a random unvisited target (can replace with frontier-based planner)
    target = random.choice(unvisited)

    # Move toward it (teleport for simplicity — later replace with path planning)
    event = controller.step(
        action="TeleportFull",
        x=target["x"],
        y=target["y"],
        z=target["z"],
        rotation={"x": 0, "y": 0, "z": 0},
        horizon=0,
        standing=True
    )
    i = save_img(i, event)
    if not event.metadata["lastActionSuccess"]:
        continue
    
    for _ in range(3):
        event = controller.step(action="RotateRight", degrees=np.random.randint(180))
        i = save_img(i, event)


print(f"Exploration complete! Final coverage: {coverage:.2%}")

controller.stop()


{'objects': [{'name': 'Apple|surface|6|28', 'position': {'x': 7.9029083251953125, 'y': 0.9998068809509277, 'z': 6.220105171203613}, 'rotation': {'x': 0.0, 'y': 180.0, 'z': 0.0}, 'visible': False, 'isInteractable': False, 'receptacle': False, 'toggleable': False, 'isToggled': False, 'breakable': False, 'isBroken': False, 'canFillWithLiquid': False, 'isFilledWithLiquid': False, 'fillLiquid': None, 'dirtyable': False, 'isDirty': False, 'canBeUsedUp': False, 'isUsedUp': False, 'cookable': False, 'isCooked': False, 'temperature': 'RoomTemp', 'isHeatSource': False, 'isColdSource': False, 'sliceable': True, 'isSliced': False, 'openable': False, 'isOpen': False, 'openness': 0.0, 'pickupable': True, 'isPickedUp': False, 'moveable': False, 'mass': 0.20000000298023224, 'salientMaterials': ['Food'], 'receptacleObjectIds': None, 'distance': 4.811234474182129, 'objectType': 'Apple', 'objectId': 'Apple|surface|6|28', 'assetId': 'Apple_15', 'parentReceptacles': ['CounterTop|6|0'], 'controlledObjects':

In [9]:
import numpy as np

np.random.randint(180)

92